In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Load dataset
data = pd.read_csv('/content/drive/MyDrive/dataset/data/food_generic_only.csv')  # Ganti 'dataset.csv' dengan nama file dataset Anda
# Preprocessing data
# data['calories'] = data['calories'].apply(lambda x: x.replace(' kcal', '')).astype(float)
# data['fat'] = data['fat'].apply(lambda x: x.replace('g', '')).astype(float)
# data['sugar'] = data['sugar'].apply(lambda x: x.replace('g', '')).astype(float)

# Calculate calories per unit
data['calories_per_unit'] = data['calories'] / data['metric_serving_amount']

from sklearn.preprocessing import MinMaxScaler

# Normalize "metric_serving_amount" with consideration to calories per unit
scaler = MinMaxScaler()
data['metric_serving_amount_normalized'] = scaler.fit_transform(data[['metric_serving_amount']])

# Adjust calories based on the normalized serving amount
data['calorie_normalized'] = data['metric_serving_amount_normalized'] * data['calories_per_unit']

def determine_meal_type(row):
    if row['calories'] <= 300 :
        return 'Dinner'
    elif row['calories'] <= 500 :
        return 'Breakfast'
    else:
        return 'Lunch'

# Create 'meal_type' column
data['meal_type'] = data.apply(determine_meal_type, axis=1)

# Save updated dataset
data.to_csv('dataset_with_meal_type.csv', index=False)

# Split data into features and labels
X = data[['calorie_normalized']]
y = data[['meal_type', 'food_name','food_id','serving_id']]  # Menambahkan kolom 'food_name' sebagai label

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize data
X_train_norm = (X_train - X_train.mean()) / X_train.std()
X_test_norm = (X_test - X_train.mean()) / X_train.std()

# Convert labels to one-hot encoding
y_train_encoded = pd.get_dummies(y_train['meal_type'])
y_test_encoded = pd.get_dummies(y_test['meal_type'])

# Define model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, activation='relu', input_shape=(1,)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train_norm, y_train_encoded, epochs=50, batch_size=32, verbose=1)

# Evaluate model
loss, accuracy = model.evaluate(X_test_norm, y_test_encoded, verbose=0)
print(f'Accuracy: {accuracy}')



Epoch 1/50
807/807 [==============================] - 8s 3ms/step - loss: 0.1105 - accuracy: 0.9717
Epoch 2/50
807/807 [==============================] - 2s 3ms/step - loss: 0.0198 - accuracy: 0.9936
Epoch 3/50
807/807 [==============================] - 2s 3ms/step - loss: 0.0129 - accuracy: 0.9957
Epoch 4/50
807/807 [==============================] - 2s 3ms/step - loss: 0.0113 - accuracy: 0.9955
Epoch 5/50
807/807 [==============================] - 2s 3ms/step - loss: 0.0102 - accuracy: 0.9960
Epoch 6/50
807/807 [==============================] - 3s 3ms/step - loss: 0.0085 - accuracy: 0.9968
Epoch 7/50
807/807 [==============================] - 2s 3ms/step - loss: 0.0081 - accuracy: 0.9969
Epoch 8/50
807/807 [==============================] - 2s 3ms/step - loss: 0.0077 - accuracy: 0.9967
Epoch 9/50
807/807 [==============================] - 2s 3ms/step - loss: 0.0089 - accuracy: 0.9964
Epoch 10/50
807/807 [==============================] - 2s 3ms/step - loss: 0.0078 - accuracy: 0.9969

In [3]:
# User input for maximum nutrient values per day
max_calorie = float(input("Enter maximum calorie per day: "))
# max_fat = float(input("Enter maximum fat per day: "))
# max_sugar = float(input("Enter maximum sugar per day: "))

# Normalize user input
user_input = pd.DataFrame([[max_calorie]], columns=['calorie_normalized'])
user_input_norm = (user_input - X_train.mean()) / X_train.std()

# Get meal recommendations
recommendations = model.predict(user_input_norm)
meal_types = y_train_encoded.columns.values

for i, recommendation in enumerate(recommendations[0]):
    print(f'Recommended {meal_types[i]}:')
    recommended_foods = y_train[(y_train['meal_type'] == meal_types[i])][['food_id', 'serving_id', 'food_name']]
    recommended_food_samples = recommended_foods.sample(n=5)
    for index, row in recommended_food_samples.iterrows():
        food_id = row['food_id']
        serving_id = row['serving_id']
        food_name = row['food_name']
        print(f'- Food ID: {food_id}, Serving ID: {serving_id}, Name: {food_name}')

Enter maximum calorie per day: 2000
1/1 [==============================] - 0s 83ms/step
Recommended Breakfast:
- Food ID: 2804, Serving ID: 11860, Name: Pork Sandwich
- Food ID: 39732, Serving ID: 40338, Name: Rice Bran (Crude)
- Food ID: 5074, Serving ID: 19549, Name: Meat Filled Pasta with Gravy (Canned)
- Food ID: 1158, Serving ID: 2051, Name: Chocolate Soft Serve Light Ice Cream Cone
- Food ID: 37583, Serving ID: 36327, Name: Beef Ribs (Large End, Trimmed to 1/8" Fat, Prime Grade, Cooked, Roasted)
Recommended Dinner:
- Food ID: 6228, Serving ID: 23499, Name: Bean Sprouts (Soybean or Mung)
- Food ID: 36625, Serving ID: 34598, Name: Taro (Without Salt, Cooked)
- Food ID: 36845, Serving ID: 34953, Name: Peas and Onions (with Salt, Frozen, Drained, Cooked, Boiled)
- Food ID: 2460, Serving ID: 9706, Name: Beef, Rice and Vegetables in Gravy (Mixture)
- Food ID: 38475, Serving ID: 37931, Name: Lamb Shoulder Blade (Trimmed to 1/4" Fat, Choice Grade, Cooked, Braised)
Recommended Lunch:
- Fo